In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import plotly.express as px


In [3]:
#definindo estados do nordeste e suas siglas referentes
etiquetas = ['AC','AL','AP', 'AM', 'BA-1', 'BA-2','CE','DF','ES','GO','MA','MT','MS','MG-1','MG-2','PA','PB','PR','PE','PI','RJ-1', 'RJ-2','RN','RS-1', 'RS-2','RO','RR','SC-1', 'SC-2', 'SP-1','SP-2', 'SP-3', 'SP-4', 'SP-5', 'SE', 'TO']
ne = ['ALAGOAS', 'BAHIA', 'CEARÁ', 'SERGIPE', 'RIO GRANDE DO NORTE', 'PERNAMBUCO', 'PIAUÍ', 'MARANHÃO', 'PARAÍBA']


1.0.  importing and knowing dataset



In [4]:
#1st parte dos dados
data_br =pd.DataFrame()
for sigla in etiquetas[0:18]:
    data_br = pd.concat([data_br ,(pd.read_csv('https://s3-sa-east-1.amazonaws.com/ckan.saude.gov.br/dados-{}.csv'.format(sigla.lower()), encoding='latin-1', sep=';', low_memory=False,  error_bad_lines=False))], axis=0, ignore_index=True)


In [103]:
data_br.shape

(9080862, 30)

In [104]:
data_br.estado.unique()

array(['ACRE', 'AMAZONAS', 'BAHIA', 'RIO GRANDE DO SUL', 'PARANÁ',
       'SÃO PAULO', 'PIAUÍ', 'CEARÁ', 'RONDÔNIA', 'MINAS GERAIS', 'GOIÁS',
       'DISTRITO FEDERAL', 'RIO DE JANEIRO', 'MATO GROSSO', 'RORAIMA',
       'PARAÍBA', 'SANTA CATARINA', 'MARANHÃO', 'PARÁ', 'SERGIPE',
       'PERNAMBUCO', 'MATO GROSSO DO SUL', 'ESPÍRITO SANTO',
       'RIO GRANDE DO NORTE', 'ALAGOAS', nan, 'TOCANTINS', 'AMAPÁ'],
      dtype=object)

In [5]:
data_br =  data_br.loc[data_br.estado.isin(ne)]
data_br.estado.unique()

array(['BAHIA', 'PIAUÍ', 'CEARÁ', 'PARAÍBA', 'MARANHÃO', 'SERGIPE',
       'PERNAMBUCO', 'RIO GRANDE DO NORTE', 'ALAGOAS'], dtype=object)

In [6]:
#2st partes dos dados
for sigla in etiquetas[18:36]:
    data_br = pd.concat([data_br ,(pd.read_csv('https://s3-sa-east-1.amazonaws.com/ckan.saude.gov.br/dados-{}.csv'.format(sigla.lower()), encoding='latin-1', sep=';', low_memory=False, error_bad_lines=False))], axis=0, ignore_index=True)
data_br.shape

b'Skipping line 592258: expected 30 fields, saw 32\n'


(14392792, 30)

In [8]:
data_br =  data_br.loc[data_br.estado.isin(ne)]
data_br.estado.unique()

array(['BAHIA', 'PIAUÍ', 'CEARÁ', 'PARAÍBA', 'MARANHÃO', 'SERGIPE',
       'PERNAMBUCO', 'RIO GRANDE DO NORTE', 'ALAGOAS'], dtype=object)

In [7]:
#maioria que teve sindrome respiratória foram os técnicos ou auxiliares de enfermagem
data_br.cbo.value_counts()

3222 - TÉCNICO OU AUXILIAR EM ENFERMAGEM                             332661
2235 - ENFERMEIRO                                                    154278
225 - MÉDICO                                                         130095
515105 - AGENTE COMUNITÁRIO DE SAÚDE                                  75708
3222 - TÉCNICOS E AUXILIARES DE ENFERMAGEM                            61169
                                                                      ...  
PROFESSORA                                                                1
DESEMPREGADO                                                              1
REPRESENTANTE COMERCIAL                                                   1
2429 - PROFISSIONAIS DA INTELIGÊNCIA                                      1
7653 - OPERADORES DE MÁQUINAS NA CONFECÇÃO DE ARTEFATOS DE  COURO         1
Name: cbo, Length: 735, dtype: int64

In [ ]:
data_br.resultadoTeste.unique()

In [84]:
#casos de covid com resultado positivo e estado dos nordestes
covid_ne = pd.DataFrame()
covid_ne = data_br.loc[data_br.resultadoTeste == 'Positivo']
covid_ne = covid_ne.loc[covid_ne.dataInicioSintomas != 'undefined']
print(covid_ne.estado.unique())
print(covid_ne.resultadoTeste.unique())

['PIAUÍ' 'CEARÁ' 'BAHIA' 'PARAÍBA' 'MARANHÃO' 'ALAGOAS' 'PERNAMBUCO'
 'SERGIPE' 'RIO GRANDE DO NORTE']
['Positivo']


In [115]:
#data_br.shape = (14.392.792, 30)
covid_ne.shape



(1383243, 9)

In [26]:
covid_ne.columns

Index(['inicioSintomas', 'profissional', 'cbo', 'resultado', 'sexo', 'estado',
       'idade', 'encerramento', 'evolucao'],
      dtype='object')

In [87]:
#selecionando colunas que iremos usar
covid_ne = covid_ne.iloc[:,[0,2,5,6,11,13,14,26,27,28]].copy() #choose columns
covid_ne.columns = ['id', 'inicioSintomas', 'profissional','cbo', 'resultado', 'sexo', 'estado', 'idade', 'encerramento', 'evolucao']

#usando id como index
covid_ne.index =covid_ne.id
covid_ne.drop(labels="id",axis=1,inplace=True) 





In [88]:
def formatDate(date_):
  day = date_[0:10]
  day_dt = pd.to_datetime(day,errors='ignore')      
  return day_dt
  



In [89]:
#como iremos usar em série temporal:
covid_ne.inicioSintomas = covid_ne.inicioSintomas.apply(formatDate)


In [182]:
covid_ne.sort_values(by=['inicioSintomas'], ascending=False)
covid_ne.loc[covid_ne.inicioSintomas < pd.datetime.now()] #ordenadno por data
covid_ne  = covid_ne.loc[(covid_ne.inicioSintomas <= pd.datetime.now()) & (covid_ne.inicioSintomas >= pd.to_datetime('2020-02-01'))].copy() # eleiminando dados com datas após o dia atual


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [220]:
ne_date = covid_ne.loc[covid_ne.profissional == 'Sim', ['inicioSintomas', 'estado', 'evolucao']]
prof_obito = ne_date.loc[ne_date.evolucao=='Óbito',['inicioSintomas', 'estado']]
prof_obito.loc[(prof_obito.inicioSintomas == pd.to_datetime('2020-03-18'))]


,inicioSintomas,estado
id,,
Mhpk4uNhe1,2020-03-18,CEARÁ


In [222]:
#novos casos por dia de inicio de sintomas
prof_dia = ne_date[['inicioSintomas', 'estado']].groupby(by = ['inicioSintomas'])['estado'].value_counts().unstack().fillna(0)
prof_dia

estado,ALAGOAS,BAHIA,CEARÁ,MARANHÃO,PARAÍBA,PERNAMBUCO,PIAUÍ,RIO GRANDE DO NORTE,SERGIPE
inicioSintomas,,,,,,,,,
2020-02-01,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0
2020-02-02,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2020-02-03,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-02-05,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0
2020-02-08,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
2020-10-13,0.0,39.0,5.0,0.0,5.0,2.0,10.0,0.0,0.0
2020-10-14,1.0,30.0,3.0,6.0,5.0,1.0,6.0,0.0,1.0
2020-10-15,1.0,13.0,0.0,1.0,3.0,0.0,4.0,0.0,2.0


In [219]:
#obitos por estados por dia
ob_dia = prof_obito.groupby(by = ['inicioSintomas'])['estado'].value_counts().unstack().fillna(0)
ob_dia

estado,ALAGOAS,BAHIA,CEARÁ,MARANHÃO,PARAÍBA,PERNAMBUCO,PIAUÍ,RIO GRANDE DO NORTE
inicioSintomas,,,,,,,,
2020-03-18,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2020-03-26,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2020-03-27,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-03-29,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2020-03-30,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2020-08-20,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-08-21,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-09-04,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0


In [242]:
ne_states = ['Alagoas', 'Bahia', 'Ceará', 'Sergipe', 'Rio Grande do Norte', 'Pernambuco', 'Piauí', 'Maranhão', 'Paraíba']

In [292]:
# Using plotly.express
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


r = 1
for i, state in enumerate(ne_states):
  fig.add_trace(go.Scatter(x=prof_dia.index, y=prof_dia[state.upper()]),
              row=r, col=(i%3)+1)
  if (((i%3)+1) == 3):
    r += 1

  
fig.update_layout(showlegend=False)
fig.show()

inicioSintomas
2020-02-01    0.0
2020-02-02    0.0
2020-02-03    0.0
2020-02-05    0.0
2020-02-08    0.0
             ... 
2020-10-13    0.0
2020-10-14    1.0
2020-10-15    1.0
2020-10-16    0.0
2020-10-17    0.0
Name: ALAGOAS, Length: 256, dtype: float64
inicioSintomas
2020-02-01     0.0
2020-02-02     0.0
2020-02-03     0.0
2020-02-05     0.0
2020-02-08     0.0
              ... 
2020-10-13    39.0
2020-10-14    30.0
2020-10-15    13.0
2020-10-16     8.0
2020-10-17     1.0
Name: BAHIA, Length: 256, dtype: float64
inicioSintomas
2020-02-01    1.0
2020-02-02    0.0
2020-02-03    1.0
2020-02-05    2.0
2020-02-08    1.0
             ... 
2020-10-13    5.0
2020-10-14    3.0
2020-10-15    0.0
2020-10-16    2.0
2020-10-17    0.0
Name: CEARÁ, Length: 256, dtype: float64
inicioSintomas
2020-02-01    0.0
2020-02-02    0.0
2020-02-03    0.0
2020-02-05    0.0
2020-02-08    0.0
             ... 
2020-10-13    0.0
2020-10-14    1.0
2020-10-15    2.0
2020-10-16    0.0
2020-10-17    0.0
Name: SERGIP

In [293]:
fig.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import numpy as np

# Initialize figure with 4 3D subplots
fig = make_subplots(
    rows=3, cols=3,
    specs=[[{'type': 'surface'}, {'type': 'surface'}, {'type': 'surface'}],
           [{'type': 'surface'}, {'type': 'surface'}, {'type': 'surface'}],
           [{'type': 'surface'}, {'type': 'surface'}, {'type': 'surface'}]])

# Generate data
x = np.linspace(-5, 60, 10)
y = np.linspace(-5, 60, 10)
xGrid, yGrid = np.meshgrid(y, x)
z = xGrid ** 3 + yGrid ** 3

# adding surfaces to subplots.
fig.add_trace(
    go.Surface(x=x, y=y, z=z, colorscale='Viridis', showscale=False),
    row=1, col=1)

fig.add_trace(
    go.Surface(x=x, y=y, z=z, colorscale='RdBu', showscale=False),
    row=1, col=2)

fig.add_trace(
    go.Surface(x=x, y=y, z=z, colorscale='RdBu', showscale=False),
    row=1, col=3)

fig.add_trace(
    go.Surface(x=x, y=y, z=z, colorscale='YlOrRd', showscale=False),
    row=2, col=1)

fig.add_trace(
    go.Surface(x=x, y=y, z=z, colorscale='YlGnBu', showscale=False),
    row=2, col=2)

fig.add_trace(
    go.Surface(x=x, y=y, z=z, colorscale='YlGnBu', showscale=False),
    row=2, col=3)

fig.add_trace(
    go.Surface(x=x, y=y, z=z, colorscale='YlOrRd', showscale=False),
    row=3, col=1)

fig.add_trace(
    go.Surface(x=x, y=y, z=z, colorscale='YlGnBu', showscale=False),
    row=3, col=2)

fig.add_trace(
    go.Surface(x=x, y=y, z=z, colorscale='YlGnBu', showscale=False),
    row=3, col=3)


fig.update_layout(
    title_text='3D subplots with different colorscales',
    height=800,
    width=800
)

fig.show()
